In [1]:
from fastai.vision.all import *

In [2]:
pd.options.display.max_columns = 100

In [3]:
datapath = Path("/../rsna_data/")
embspath = Path(datapath/"cnn_embs")
train_df = pd.read_csv(datapath/'train.csv')

In [4]:
list(embspath.ls())

[Path('/../rsna_data/cnn_embs/full_effb3_256_FOLD0'),
 Path('/../rsna_data/cnn_embs/full_effb3_512_FOLD0'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD1'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD2'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD3'),
 Path('/../rsna_data/cnn_embs/.ipynb_checkpoints'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD0'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD4')]

### Load Embeddings & Preds

In [5]:
fold = 0
folddirs = [embspath/f'full_512_FOLD{fold}']; folddirs

[Path('/../rsna_data/cnn_embs/full_512_FOLD0')]

In [6]:
[o.name for o in folddirs[0].ls()]

['files.pth',
 'seq_img_preds.pth',
 'preds.pth',
 'pids.pth',
 'seq_exam_preds.pth',
 'embeddings.pth']

In [7]:
embeddings = torch.cat([torch.load(folddir/'embeddings.pth') for folddir in folddirs])
preds = torch.cat([torch.load(folddir/'preds.pth') for folddir in folddirs])
files = []
for folddir in folddirs: files += torch.load(folddir/'files.pth')

In [8]:
embeddings.shape, preds.shape, len(files)

(torch.Size([358687, 1024]), torch.Size([358687, 2]), 358687)

In [9]:
# add zero for padded input idx
input_pad_idx = len(embeddings)
embeddings = torch.cat([embeddings, torch.zeros_like(embeddings[:1])])

In [10]:
embeddings[input_pad_idx], embeddings.shape, input_pad_idx

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), torch.Size([358688, 1024]), 358687)

In [11]:
type(embeddings)

torch.Tensor

### Metadata Features

In [12]:
metadata_path = datapath/'metadata'
metadata_files = get_files(metadata_path, extensions=".csv")
metadf = pd.concat([pd.read_csv(o) for o in metadata_files]).reset_index(drop=True)

In [13]:
metadf.shape

(1790594, 68)

In [14]:
len(metadf['StudyInstanceUID'].unique())

7279

In [15]:
def minmax_scaler(o): return (o - min(o))/(max(o) - min(o))

In [16]:
scaled_pos = metadf.groupby('StudyInstanceUID')['ImagePositionPatient2'].apply(minmax_scaler)
metadf.loc[:,'scaled_position'] = scaled_pos.values

In [17]:
meta_feat_cols = ['scaled_position']

In [18]:
assert np.isnan(metadf[meta_feat_cols]).sum().sum() == 0

In [19]:
mean_std = metadf[meta_feat_cols].agg(['mean', 'std']).T

In [20]:
mean_std_dict = dict(zip(mean_std.index, mean_std.values.tolist())); mean_std_dict

{'scaled_position': [0.5078721739409284, 0.29139548181397823]}

In [21]:
# standard scaler for training
for c in mean_std_dict: metadf[c] = (metadf[c] - mean_std_dict[c][0]) / mean_std_dict[c][1]

In [22]:
meta_feats_dict = dict(zip(metadf['SOPInstanceUID'], metadf[meta_feat_cols].to_numpy()))

In [23]:
len(meta_feats_dict)

1790594

### Fold Metadata

In [24]:
meta_embeddings = []
for o in files:
    sopid = o.stem.split("_")[1]
    meta_embeddings.append(meta_feats_dict[sopid])
meta_embeddings = np.vstack(meta_embeddings)
meta_embeddings= tensor(meta_embeddings)

In [25]:
meta_embeddings.shape, type(meta_embeddings)

(torch.Size([358687, 1]), torch.Tensor)

In [26]:
meta_embeddings = torch.cat([meta_embeddings, torch.zeros_like(meta_embeddings[:1])])

In [27]:
embeddings.shape, meta_embeddings.shape

(torch.Size([358688, 1024]), torch.Size([358688, 1]))

In [28]:
combined_embeddings = torch.cat([embeddings, meta_embeddings], 1)

In [29]:
combined_embeddings.shape

torch.Size([358688, 1025])

In [30]:
combined_embeddings

tensor([[ 9.6094, 11.6719,  9.2188,  ...,  0.5684,  0.2417, -0.1343],
        [ 4.4648, 11.4766,  5.2500,  ...,  0.3477,  0.2727,  0.9382],
        [ 4.9805,  3.4980,  0.0000,  ...,  0.4731,  0.2737,  1.5816],
        ...,
        [12.5312,  1.9893, 10.1328,  ...,  0.3242,  1.5664, -0.7226],
        [ 8.1797,  2.8672, 18.1094,  ...,  0.6055,  1.2500, -1.0009],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

### Data

In [31]:
from fastai.text.all import *

In [32]:
image_targets = ['pe_present_on_image']
exam_targets = [
    'negative_exam_for_pe', # exam level
    'rv_lv_ratio_gte_1', # exam level
    'rv_lv_ratio_lt_1', # exam level
    'leftsided_pe', # exam level
    'chronic_pe', # exam level
    'rightsided_pe', # exam level
    'acute_and_chronic_pe', # exam level
    'central_pe', # exam level
    'indeterminate' # exam level
]

In [33]:
targets_df = train_df[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']+image_targets+exam_targets]

In [34]:
targets_dict = dict(zip(targets_df['SOPInstanceUID'].values, targets_df[image_targets+exam_targets].values))

In [35]:
len(targets_dict)

1790594

In [36]:
files_dict = defaultdict(list)
for i,o in enumerate(files):
    slice_no, sopid = o.stem.split("_")
    sid = o.parent.name
    slice_no = int(slice_no)        
    files_dict[sid].append({"slice_no":slice_no, "embs_idx":i, "img_y":targets_dict[sopid][0], "exam_y":targets_dict[sopid][1:]})

In [37]:
all_pids = list(files_dict.keys())

In [38]:
len(all_pids)

1456

In [39]:
def get_x(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    return tensor([o['embs_idx'] for o in l])

def get_img_y(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    img_y = [o['img_y'] for o in l]
    img_y = tensor(img_y).float()
    return img_y

def get_exam_y(pid, files_dict):
    d = files_dict[pid][0]        
    exam_y = tensor(d['exam_y']).float()
    return exam_y
    
# before_batch: after collecting samples before collating
targ_pad_idx = 666
def SequenceBlock():       return  TransformBlock(type_tfms=[partial(get_x, files_dict=files_dict)], 
                                                  dl_type=SortedDL,
                                                  dls_kwargs={'before_batch':
                                                               [partial(pad_input, pad_idx=input_pad_idx),
                                                                partial(pad_input, pad_idx=targ_pad_idx, pad_fields=1)]})
def SequenceTargetBlock(): return TransformBlock(type_tfms=[partial(get_img_y, files_dict=files_dict)])
def TargetBlock():         return TransformBlock(type_tfms=[partial(get_exam_y, files_dict=files_dict)])

In [40]:
get_img_y(all_pids[0], files_dict)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [41]:
get_exam_y(all_pids[0], files_dict)

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [42]:
# normalized_embeddings = F.normalize(combined_embeddings, dim=0)
# normalized_embeddings.isnan().sum()
# normalized_embeddings

In [43]:
assert combined_embeddings.isnan().sum().item() == 0

### Model

In [44]:
device = default_device(); device

device(type='cuda', index=0)

In [45]:
class AWD_LSTM(Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182"
    initrange=0.1

    def __init__(self, emb_sz,n_hid, n_layers, hidden_p=0.2, input_p=0.6, weight_p=0.5, bidir=False):
        store_attr('emb_sz,n_hid,n_layers')
        self.bs = 1
        self.n_dir = 2 if bidir else 1
        
        self.rnns = nn.ModuleList([self._one_rnn(emb_sz if l == 0 else n_hid, (n_hid)//self.n_dir, bidir, weight_p, l) for l in range(n_layers)])

        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])
        self.reset()

    def forward(self, x, from_embeds=False):
        
        if from_embeds: inp = x
        else: inp = combined_embeddings[x].to(device)
        bs,sl = inp.shape[:2]
        if bs!=self.bs: self._change_hidden(bs)

        output = self.input_dp(inp)
        new_hidden = []
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            output, new_h = rnn(output, self.hidden[l])
            new_hidden.append(new_h)
            if l != self.n_layers - 1: output = hid_dp(output)
        self.hidden = to_detach(new_hidden, cpu=False, gather=False)
        return output

    def _change_hidden(self, bs):
        self.hidden = [self._change_one_hidden(l, bs) for l in range(self.n_layers)]
        self.bs = bs

    def _one_rnn(self, n_in, n_out, bidir, weight_p, l):
        "Return one of the inner rnn"
        rnn = nn.LSTM(n_in, n_out, 1, batch_first=True, bidirectional=bidir, bias=False)
        return WeightDropout(rnn, weight_p)

    def _one_hidden(self, l):
        "Return one hidden state"
        nh = (self.n_hid) // self.n_dir
        return (one_param(self).new_zeros(self.n_dir, self.bs, nh), one_param(self).new_zeros(self.n_dir, self.bs, nh))

    def _change_one_hidden(self, l, bs):
        if self.bs < bs:
            nh = (self.n_hid) // self.n_dir
            return tuple(torch.cat([h, h.new_zeros(self.n_dir, bs-self.bs, nh)], dim=1) for h in self.hidden[l])
        if self.bs > bs: return (self.hidden[l][0][:,:bs].contiguous(), self.hidden[l][1][:,:bs].contiguous())
        return self.hidden[l]

    def reset(self):
        "Reset the hidden states"
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.hidden = [self._one_hidden(l) for l in range(self.n_layers)]

In [46]:
layers = [512 * 3] + [512] + [9]

class MultiHeadedSequenceClassifier(Module):
    "dim: input sequence feature dim"
    def __init__(self, bptt=72, input_pad_idx=input_pad_idx, n_meta=1, dim=1024):
        
        store_attr('input_pad_idx')
        self.awd_lstm = AWD_LSTM(dim+n_meta, 512, 2, bidir=True)
        self.encoder = SentenceEncoder(bptt=bptt, module=self.awd_lstm, pad_idx=input_pad_idx)
        
        # image level preds
        self.seq_head = LinearDecoder(1, 512, bias=True)
 
        # exam level preds
        self.exam_head = PoolingLinearClassifier(layers, ps=[0.4, 0.1], bptt=bptt)
        
    
    def forward(self, x):
        out, mask = self.encoder(x) 
       
        # img level out
        seq_cls_out,_,_ = self.seq_head(out)
        seq_cls_out = seq_cls_out.squeeze(-1)
              
        # exam level out
        exam_out,_,_ = self.exam_head((out,mask))

        return (seq_cls_out, exam_out)

In [47]:
class MultiLoss(Module):
    
    def __init__(self, targ_pad_idx=666):
        store_attr("targ_pad_idx")

    def forward(self, inp, yb0, yb1):
        exam_target_weights = tensor([0.0736196319, 0.2346625767,  0.0782208589, 
                                      0.06257668712, 0.1042944785, 0.06257668712,
                                      0.1042944785,  0.1877300613, 0.09202453988]).to(yb1.device)
        seq_cls_out, exam_out = inp
       
        # img loss
        mask = yb0 != self.targ_pad_idx 
        
        img_loss, qs = 0, 0        
        for _m,_y,_p in zip(mask, yb0, seq_cls_out):
            qi = _y[_m].mean()
            qs += qi*sum(_m)
            img_loss += qi*(F.binary_cross_entropy_with_logits(_p[_m], _y[_m], reduction='sum'))
        
        # exam loss
        exam_losses = F.binary_cross_entropy_with_logits(exam_out, yb1,reduction='none')
        tot_exam_loss = (exam_losses*(exam_target_weights.unsqueeze(0))).sum()
        tot_exam_wgts = len(exam_losses)*(tensor(exam_target_weights).sum())
        
        return (tot_exam_loss+img_loss)/(qs+tot_exam_wgts)

### Train

In [48]:
data = DataBlock(blocks=(SequenceBlock,SequenceTargetBlock,TargetBlock), 
                 n_inp=1, 
                 splitter=FuncSplitter(lambda o: False)
#                 splitter=RandomSplitter(0.3),
                )
dls = data.dataloaders(all_pids, bs=64)
model = SequentialRNN(MultiHeadedSequenceClassifier(bptt=256))
loss_func = MultiLoss()
learner = Learner(dls, model, loss_func=loss_func, metrics=[], cbs=[ModelResetter(), TerminateOnNaNCallback()])

In [313]:
# learner.lr_find()

In [314]:
# learner.fit_flat_cos(40, lr=0.005)

In [315]:
# learner.save(f"nometa_sequence_model_wider_fold{fold}", with_opt=False)

### Get Preds

In [49]:
model_fold = 1
assert fold != model_fold

In [50]:
learner.load(f"nometa_sequence_model_fold{model_fold}");

In [51]:
learner.model.eval().to(device);

In [52]:
len(all_pids)

1456

In [53]:
stacking_path = datapath/'stacking'

In [54]:
stacking_dir = stacking_path/f'DATA_FOLD{fold}_FROM1234'

In [55]:
if not stacking_dir.exists(): stacking_dir.mkdir()

In [57]:
test_dl = learner.dls.test_dl(all_pids, with_labels=True)

In [64]:
xb,yb0,yb1 = test_dl.one_batch()

In [70]:
seq_img_preds = []
seq_exam_preds = []
with torch.no_grad():
    for xb, yb0, yb1 in progress_bar(test_dl):
        img_pred, exam_pred = learner.model(xb)
        seq_img_preds.append(img_pred)
        seq_exam_preds.append(exam_pred)
        loss = loss_func((img_pred, exam_pred), yb0, yb1)
        break

In [71]:
loss

tensor(0.9774, device='cuda:0')

In [67]:
yb1

tensor([[0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0

In [355]:
torch.save(seq_img_preds, stacking_dir/f'seq_img_preds_model_fold{model_fold}.pth')
torch.save(seq_exam_preds, stacking_dir/f'seq_exam_preds_model_fold{model_fold}.pth')

In [356]:
torch.save(all_pids, stacking_dir/f'pids.pth')